                    # Homework 1                

# Q1. Running Elastic
Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:
```python
curl localhost:9200
```
What's the version.build_hash value?

**Ans** :  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73'

In [1]:
# Get data
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': 'd4c4dd3b1785', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ZeznkbE2SWmmnnaXqj4WSQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Which function do you use for adding your data to elastic?

- insert
- index
- put
- add

**Ans** : index

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
from tqdm.auto import tqdm

In [5]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

* 84.50
* 64.50
* 44.50
* 24.50

Look at the _score field.

**Ans** : 44.55304

In [6]:
query = "How do execute a command on a Kubernetes pod?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)

In [7]:
search_results['hits']['hits'][0]['_score']

44.50556

# Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

**Ans** : How do I copy files from a different folder into docker container’s working directory?

In [8]:
query='How do copy a file to a Docker container?'

search_query_filtered = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
            "term": {
                "course": "machine-learning-zoomcamp"
            }
        }
        }
    }
}

search_results_filtered = es_client.search(index=index_name, body=search_query)

In [9]:
search_results_filtered['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'soiprZcBA1FVp5hVYk_s',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'QYiprZcBA1FVp5hVgFAE',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '0oiprZcBA1FVp5hVaU97',
  '_score': 33.70974,
  '_source': {'text': 'You 

# Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context.

Separate context entries by two linebreaks (\n\n)


Now use the context you just created along with the "How do copy a file to a Docker container?" question to construct a prompt using the template below:

```python
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the len function)
- 946
- 1446
- 1946
- 2446

**Ans** : 1917 =~ 1946

In [10]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [11]:
context_pieces=[]

for hit in search_results_filtered['hits']['hits']:
    doc=hit['_source']
    context_pice=context_template.format(**doc)
    context_pieces.append(context_pice)

context = '\n\n'.join(context_pieces)

In [12]:
prompt = prompt_template.format(question=query, context=context)

In [13]:
len(prompt)

1917

# Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

```python

pip install tiktoken
```

Let's calculate the number of tokens in our query:

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the encode function. How many tokens does our prompt have?

- 120
- 220
- 320
- 420

Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

```python
encoding.decode_single_token_bytes(63842)

**Ans** : 448 =~ 420

In [14]:
import tiktoken

In [15]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [16]:
len(encoding.encode(prompt))


448

In [17]:
tokens = encoding.encode(prompt)[:10]
tokens

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122]

In [18]:
encoding.decode_single_token_bytes(tokens[5])


b'.'